In [ ]:
import numpy as np
import pandas as pd
!pip install beautifulsoup4
import os
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Progress Bar
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import transformers
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import warnings
from transformers import logging as hf_logging
hf_logging.set_verbosity_error() # Hidding Huggingface Warnings
warnings.filterwarnings("ignore")


In [ ]:
MODEL_NAME = 'distilbert-base-cased'


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
df = pd.read_csv('Train.csv')

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
def convert_to_lower(text):
    return text.lower()
def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

import string
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    removed = []
    stop_words = list(stopwords.words("english"))
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)
def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc
def lemmatizing(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        lemma_word = lemmatizer.lemmatize(tokens[i])
        tokens[i] = lemma_word
    return " ".join(tokens)

In [ ]:
# Concatenate columns as strings
df['Combined'] = df['post_text'].astype(str) + df['pre_text'].astype(str) + df['table'].astype(str)

df['Combined_clean'] = df['Combined'].apply(lambda x: convert_to_lower(x))
df['Lenght_before'] = df['Combined_clean'].apply(lambda x: len(x))

In [ ]:
df['Combined_clean'] = df['Combined_clean'].apply(lambda x: remove_numbers(x))
df['Combined_clean'] = df['Combined_clean'].apply(lambda x: remove_punctuation(x))
df['Combined_clean'] = df['Combined_clean'].apply(lambda x: remove_stopwords(x))
df['Combined_clean'] = df['Combined_clean'].apply(lambda x: remove_extra_white_spaces(x))
df['Lenght_after'] = df['Combined_clean'].apply(lambda x: len(x))


In [ ]:
le = LabelEncoder()
df['industry_label'] = le.fit_transform(df['industry'])

In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer


sid = SentimentIntensityAnalyzer()

# Perform sentiment analysis on each text in the dataset
sentiments = []
for text in df['text']:
    # Get the polarity scores
    scores = sid.polarity_scores(text)

    # Determine the sentiment label based on the compound score
    if scores['compound'] >= 0.05:
        sentiment = 'Positive'
    elif scores['compound'] <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    # Append the sentiment label to the list
    sentiments.append(sentiment)

# Add the sentiment labels to the dataframe
df['sentiment'] = sentiments

# Print the dataframe with sentiment labels
print(df.head())

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
tf_wb= TfidfVectorizer()
X_tf = tf_wb.fit_transform(df['Combined_clean'])
X_tf = X_tf.toarray()

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')



In [ ]:
import torch

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
df.dropna

<bound method DataFrame.dropna of                            id  \
0       EW/2016/page_50.pdf-2   
1     ETR/2016/page_352.pdf-3   
2      GS/2012/page_186.pdf-3   
3      ADI/2011/page_83.pdf-2   
4     PNC/2014/page_203.pdf-3   
...                       ...   
1967    GS/2015/page_76.pdf-3   
1968  JPM/2009/page_238.pdf-2   
1969   AES/2001/page_45.pdf-2   
1970  ETR/2017/page_175.pdf-3   
1971   STT/2006/page_96.pdf-4   

                                              post_text  \
0     ['( a ) the amount included in 2018 2018less t...   
1     ['see note 4 to the financial statements for a...   
2     ['rwas under the federal reserve board 2019s r...   
3     ['12 .'\n 'commitments and contingencies from ...   
4     ['the pnc financial services group , inc .' '2...   
...                                                 ...   
1967  ['1 .'\n 'net revenues related to our consolid...   
1968  ['non-u.s. ( a ) 9804 4867 9085 income before ...   
1969  ['* includes venezuela and colombia .'\n 'sell...   
1970  ['entergy corporation and subsidiaries notes t...   
1971  ['compensation expense related to stock option...   

                                               pre_text  \
0     ['net cash flows provided by operating activit...   
1     ['entergy louisiana , llc and subsidiaries man...   
2     ['notes to consolidated financial statements n...   
3     ['the following is a schedule of future minimu...   
4     ['to determine stock-based compensation expens...   
...                                                 ...   
1967  ['the goldman sachs group , inc .'\n 'and subs...   
1968  ['notes to consolidated financial statements j...   
1969  ['of sales , competitive supply gross margin d...   
1970  ['as of december a031 , 2017 , system energy ,...   
1971  ['for the year ended december 31 , 2005 , we r...   

                                                  table        industry  \
0     [array(['contractual obligations', 'payments d...  Pharmaceutical   
1     [array(['2016', '2015', '2014', '2013'], dtype...          Energy   
2     [array(['$ in millions', 'as of december 2012'...         Finance   
3     [array(['fiscal years', 'operating leases'], d...      Technology   
4     [array(['shares in thousands december 31 2013'...         Finance   
...                                                 ...             ...   
1967  [array(['$ in millions', 'year ended december ...         Finance   
1968  [array(['year ended december 31 ( in millions ...         Finance   
1969  [array(['north america', '2001 $ 912 million',...          Energy   
1970  [array(['', 'amount ( in thousands )'], dtype=...          Energy   
1971  [array(['', '2006', '2005', '2004'], dtype=obj...         Finance   

                                               Combined  \
0     ['( a ) the amount included in 2018 2018less t...   
1     ['see note 4 to the financial statements for a...   
2     ['rwas under the federal reserve board 2019s r...   
3     ['12 .'\n 'commitments and contingencies from ...   
4     ['the pnc financial services group , inc .' '2...   
...                                                 ...   
1967  ['1 .'\n 'net revenues related to our consolid...   
1968  ['non-u.s. ( a ) 9804 4867 9085 income before ...   
1969  ['* includes venezuela and colombia .'\n 'sell...   
1970  ['entergy corporation and subsidiaries notes t...   
1971  ['compensation expense related to stock option...   

                                         Combined_clean  Lenght_before  \
0     amount included less year reflects anticipated...           4184   
1     see note financial statements description mone...           4099   
2     rwas federal reserve board riskbased capital r...           4750   
3     commitments contingencies time time ordinary c...           2873   
4     pnc financial services group inc form determin...           5241   
...                                                 ...            ...   
1967  net revenues rela

In [ ]:
df.head()

,id,post_text,pre_text,table,industry,Combined,Combined_clean,Lenght_before,Lenght_after,industry_label
0,EW/2016/page_50.pdf-2,['( a ) the amount included in 2018 2018less t...,['net cash flows provided by operating activit...,"[array(['contractual obligations', 'payments d...",Pharmaceutical,['( a ) the amount included in 2018 2018less t...,amount included less year reflects anticipated...,4184,2542,2
1,ETR/2016/page_352.pdf-3,['see note 4 to the financial statements for a...,"['entergy louisiana , llc and subsidiaries man...","[array(['2016', '2015', '2014', '2013'], dtype...",Energy,['see note 4 to the financial statements for a...,see note financial statements description mone...,4099,2876,0
2,GS/2012/page_186.pdf-3,['rwas under the federal reserve board 2019s r...,['notes to consolidated financial statements n...,"[array(['$ in millions', 'as of december 2012'...",Finance,['rwas under the federal reserve board 2019s r...,rwas federal reserve board riskbased capital r...,4750,3197,1
3,ADI/2011/page_83.pdf-2,['12 .'\n 'commitments and contingencies from ...,['the following is a schedule of future minimu...,"[array(['fiscal years', 'operating leases'], d...",Technology,['12 .'\n 'commitments and contingencies from ...,commitments contingencies time time ordinary c...,2873,1862,3
4,PNC/2014/page_203.pdf-3,"['the pnc financial services group , inc .' '2...",['to determine stock-based compensation expens...,[array(['shares in thousands december 31 2013'...,Finance,"['the pnc financial services group , inc .' '2...",pnc financial services group inc form determin...,5241,3457,1


In [ ]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.cluster import KMeans

# Load the dataset
df = pd.read_csv('Train.csv')  # Replace 'unlabeled_data.csv' with the name of your dataset

# Preprocess the text data
# Implement your text preprocessing steps here

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the text data
tokenized_texts = [tokenizer(text, return_tensors='pt') for text in df['Combined_clean']]

# Concatenate the tokenized tensors
input_ids = torch.cat([t['input_ids'] for t in tokenized_texts], dim=0)
attention_masks = torch.cat([t['attention_mask'] for t in tokenized_texts], dim=0)

# Load the DistilBERT model
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Encode the text data
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)

# Get the embeddings
embeddings = outputs.last_hidden_state

# Perform clustering (example using KMeans)
num_clusters = 5  # Number of clusters to create
kmeans = KMeans(n_clusters=num_clusters)
cluster_labels = kmeans.fit_predict(embeddings)

# Analyze the clusters
for cluster_id in range(num_clusters):
    cluster_indices = cluster_labels == cluster_id
    cluster_texts = df.loc[cluster_indices, 'text']
    print(f"Cluster {cluster_id + 1} - Number of samples: {sum(cluster_indices)}")
    print("Sample texts:")
    print(cluster_texts.sample(min(5, len(cluster_texts))))
    print("-----")

KeyError: 'Combined_clean'

In [ ]:
le = LabelEncoder()
df_train['industry_label'] = le.fit_transform(train_df['Label'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df_train['Combined'].values, df_train['industry_label'].values, test_size=0.1, random_state=42)


In [ ]:
df_train.head()

In [ ]:
label_dict = (df_train[['industry','industry_label']].drop_duplicates()
              .sort_values(by='industry_label')
              .reset_index(drop=True)['industry']
              .to_dict())

for index, key in label_dict.items():
    print(index, key)

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')



In [ ]:
!pip install accelerate>=0.21.0
!pip install transformers>=4.11.3
!pip install transformers[torch]


In [ ]:
# Install the required libraries (if not already installed)
# !pip install transformers

import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Load your preprocessed dataset (replace with your actual data)
# Assuming you have 'X_train', 'y_train', 'X_val', and 'y_val'

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Encode the text data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True)

# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(y_train),
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings["input_ids"]),
    torch.tensor(val_encodings["attention_mask"]),
    torch.tensor(y_val),
)

# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./distilbert_model",  # Directory to save the model
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=500,
    logging_dir="./logs",
    num_train_epochs=1
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {"input_ids": torch.stack([f[0] for f in data]),
                                "attention_mask": torch.stack([f[1] for f in data]),
                                "labels": torch.stack([f[2] for f in data])},
)

# Train the model
trainer.train()

# Save the trained model
model.save_pretrained("path/to/save/directory")


{'eval_loss': 0.2651739716529846, 'eval_runtime': 209.3543, 'eval_samples_per_second': 0.946, 'eval_steps_per_second': 0.119, 'epoch': 0.45}
{'eval_loss': 0.07931794971227646, 'eval_runtime': 193.5064, 'eval_samples_per_second': 1.023, 'eval_steps_per_second': 0.129, 'epoch': 0.9}
{'train_runtime': 7094.877, 'train_samples_per_second': 0.25, 'train_steps_per_second': 0.031, 'train_loss': 0.3859457411207594, 'epoch': 1.0}


In [ ]:
import pickle

# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()

# Get the predicted labels for the validation dataset
y_pred = trainer.predict(val_dataset).predictions.argmax(axis=1)

# Get the true labels for the validation dataset
y_true = val_dataset[:][2].numpy()

# Calculate the accuracy
accuracy = accuracy_score(y_true, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)

# Load the PyTorch model
model_path = "path/to/save/directory"  # Replace with the path where your model is saved
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Save the model as a .pkl file using pickle
with open("distilbert_model.pkl", "wb") as f:
    pickle.dump(model, f)


{'eval_loss': 0.07442976534366608, 'eval_runtime': 234.0441, 'eval_samples_per_second': 0.846, 'eval_steps_per_second': 0.107, 'epoch': 1.0}
Accuracy: 0.9696969696969697


In [ ]:
text ='''Dear manager,
I recently had a payment issue while using your company. The reason I was given was "Due to energy issue, the payment servers were down for a while, we are sorry for the issue, we will ensure this won't happen again." The reason was not ok for me, I feel really unsatisfied and energy was not good. If I was travelling some where and I faced this issue, what should I do? If there is a pharmaceutical emergency?? This kind of technological issues should not be happening with such a brand, you have trust of billions of customers, thank you.
'''

# Preprocess text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted class
predicted_class = torch.argmax(outputs.logits).item()

# Map predicted class to label
labels = ["Energy", "Finance", "Pharmaceutical","Technology","Travel"]  # Replace with your actual class labels
predicted_label = labels[predicted_class]

print("Predicted Label:", predicted_label)


Predicted Label: Pharmaceutical
